In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from functions import *
import tqdm

In [2]:
dmso_csv_path = '/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/paths_to_channels_creation/paths_dmso_v1v2.csv'

In [3]:
df_dmso = pd.read_csv(dmso_csv_path)

In [4]:
df_dmso.head()

,Unnamed: 0,plate,well,neg_control_type,C1,C2,C3,C4,C5
0,0,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
1,1,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
2,2,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
3,3,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
4,4,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...


In [5]:
df_dmso = df_dmso[df_dmso["neg_control_type"] == '[dmso]']
df_dmso.plate.unique()


array(['P015076', 'P015077', 'P015078', 'P015079', 'P015080', 'P015081',
       'P015082', 'P015083', 'P015084', 'P015085', 'P015086', 'P015087',
       'P015088', 'P015089', 'P015090', 'P015091', 'P015092', 'P015093',
       'P015094', 'P015095', 'P015096', 'P015097', 'P015098', 'P015099',
       'P100980', 'P100981', 'P100982', 'P100983', 'P100984', 'P100985',
       'P100986', 'P100987', 'P100988', 'P100989', 'P100990', 'P100991',
       'P101016', 'P101017', 'P101018', 'P101019', 'P101020', 'P101021',
       'P101022', 'P101023', 'P101024', 'P101025', 'P101026', 'P101027',
       'P101028', 'P101029', 'P101030', 'P101031', 'P101032', 'P101033'],
      dtype=object)

In [33]:
test= df_dmso.plate.unique()[0:2]

In [34]:
test

array(['P015076', 'P015077'], dtype=object)

In [35]:
dmso_stats = {}

for plate in tqdm.tqdm(test):
    dmso_stats[plate] = {}
    rows = df_dmso[df_dmso.plate == plate]
    print(len(rows))
    for c in tqdm.tqdm(['C1','C2','C3','C4','C5']):
        im = []
        
        for i in range(len(rows)):
            path = rows[c].iloc[i]
            im.append(cv2.imread(path, -1))
        
        dmso_stats[plate][c] = {'m': np.mean(im), 'std':np.std(im)}

  0%|          | 0/2 [00:00<?, ?it/s]

198



 50%|█████     | 1/2 [00:48<00:48, 48.88s/it]

198



100%|██████████| 2/2 [02:28<00:00, 74.29s/it]


In [36]:
dmso_stats

{'P015076': {'C1': {'m': 298.56553513638613, 'std': 280.17719870419074},
  'C2': {'m': 1182.599314422301, 'std': 1628.216043824049},
  'C3': {'m': 1740.3659357495999, 'std': 846.2971853421564},
  'C4': {'m': 1364.0816585652478, 'std': 1029.1825133612128},
  'C5': {'m': 682.3957518471755, 'std': 704.7482234564715}},
 'P015077': {'C1': {'m': 333.64529227135034, 'std': 339.08582350494197},
  'C2': {'m': 1350.6730676892814, 'std': 1606.9584977581317},
  'C3': {'m': 2163.296099828229, 'std': 1011.646483085695},
  'C4': {'m': 1633.245623894769, 'std': 1160.5192705314935},
  'C5': {'m': 846.2881181705169, 'std': 813.9777711274285}}}

In [37]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in dmso_stats.items() for innerKey, values in innerDict.items()}

In [38]:
df_test = pd.DataFrame(reform)

In [39]:
df_test.head()

P015076                                                     \
             C1           C2           C3           C4          C5   
m    298.565535  1182.599314  1740.365936  1364.081659  682.395752   
std  280.177199  1628.216044   846.297185  1029.182513  704.748223   

        P015077                                                     
             C1           C2           C3           C4          C5  
m    333.645292  1350.673068  2163.296100  1633.245624  846.288118  
std  339.085824  1606.958498  1011.646483  1160.519271  813.977771

In [ ]:
df_test.to_csv('stats/bf_dmso_stats.csv')